# Выбор локации для скважины

Дано: пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

Задача: Построить модель машинного обучения, которая определит регион для бурения новой скважины, где добыча нефти принесёт наибольшую прибыль.

Условия:
<br>В регионе исследуют 500 точек, из которых выбирают 200 лучших для разработки.
<br>Бюджет на разработку скважин в регионе - 10 млрд рублей.
<br>Доход с одного барреля сырья - 450 рублей. Объём указан в тысячах баррелей.
<br>Оставить лишь те регионы, в которых вероятность убытков меньше 2.5%, среди них выбрать регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

### Загрузите и подготовьте данные. Поясните порядок действий

In [1]:
import pandas as pd
import numpy as np

from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

display('geo_data_0:', data0)
display('geo_data_1:', data1)
display('geo_data_2:', data2)

display('Общая информация о таблице geo_data_0:', data0.info())
display('Количество дубликатов geo_data_0:', data0.duplicated().sum())
display('Количество пропущенных значений geo_data_0:', data0.isna().sum())

display('Общая информация о таблице geo_data_1:', data1.info())
display('Количество дубликатов geo_data_1:', data1.duplicated().sum())
display('Количество пропущенных значений geo_data_1:', data1.isna().sum())

display('Общая информация о таблице geo_data_2:', data2.info())
display('Количество дубликатов geo_data_2:', data2.duplicated().sum())
display('Количество пропущенных значений geo_data_2:', data2.isna().sum())

'geo_data_0:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


'geo_data_1:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


'geo_data_2:'

...

None

'Количество дубликатов geo_data_2:'

0

'Количество пропущенных значений geo_data_2:'

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Видим, что в таблице пять столбцов, дубликатов строк и пропущенных значений нет. 
<br>
<br>Порядок действий следующий:
<br>- удалить одинаковые id, т.к. непонятно какие строки из них актуальные, при условии, что одному id соответствует одна скважина
<br>- сохранить оставшиеся id в отдельные таблицы
<br>- удалить из основных таблиц id для удобства расчетов
<br>
<br>Т.к. целевой признак количественный, решаем задачу регрессии.

In [3]:
print('Количество одинаковых id в geo_data_0 =', data0['id'].duplicated().sum())
idd0 = data0[data0['id'].duplicated()]['id']
display('Одинаковые id в geo_data_0:', idd0)
display('Строки одинаковыми id в geo_data_0:', data0.query('id in @idd0').sort_values('id'))

print('Количество одинаковых id в geo_data_1 =', data1['id'].duplicated().sum())
idd1 = data1[data1['id'].duplicated()]['id']
display('Одинаковые id в geo_data_1:', idd1)
display('Строки одинаковыми id в geo_data_1:', data1.query('id in @idd1').sort_values('id'))

print('Количество одинаковых id в geo_data_2 =', data2['id'].duplicated().sum())
idd2 = data2[data2['id'].duplicated()]['id']
display('Одинаковые id в geo_data_2:', idd2)
display('Строки одинаковыми id в geo_data_2:', data2.query('id in @idd2').sort_values('id'))

Количество одинаковых id в geo_data_0 = 10


'Одинаковые id в geo_data_0:'

7530     HZww2
41724    bxg6G
51970    A5aEY
63593    QcMuo
66136    74z30
69163    AGS9W
75715    Tdehs
90815    fiKDv
92341    TtcGQ
97785    bsk9y
Name: id, dtype: object

'Строки одинаковыми id в geo_data_0:'

,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


...

Количество одинаковых id в geo_data_2 = 4


'Одинаковые id в geo_data_2:'

43233    xCHr8
49564    VF7Jo
55967    KUPhW
95090    Vcm5J
Name: id, dtype: object

'Строки одинаковыми id в geo_data_2:'

,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


In [4]:
data0 = data0.query('id not in @idd0')
display('geo_data_0 без дублирующихся идентификаторов:', data0)

data1 = data1.query('id not in @idd1')
display('geo_data_1 без дублирующихся идентификаторов:', data1)

data2 = data2.query('id not in @idd2')
display('geo_data_2 без дублирующихся идентификаторов:', data2)

'geo_data_0 без дублирующихся идентификаторов:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


'geo_data_1 без дублирующихся идентификаторов:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


'geo_data_2 без дублирующихся идентификаторов:'

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [5]:
data0_id = data0['id']
display('Таблица geo_data_0 с уникальными идентификаторами скважины:', data0_id)

data1_id = data1['id']
display('Таблица geo_data_1 с уникальными идентификаторами скважины:', data1_id)

data2_id = data2['id']
display('Таблица geo_data_2 с уникальными идентификаторами скважины:', data2_id)

'Таблица geo_data_0 с уникальными идентификаторами скважины:'

0        txEyH
1        2acmU
2        409Wp
3        iJLyR
4        Xdl7t
         ...  
99995    DLsed
99996    QKivN
99997    3rnvd
99998    7kl59
99999    1CWhH
Name: id, Length: 99980, dtype: object

'Таблица geo_data_1 с уникальными идентификаторами скважины:'

0        kBEdx
1        62mP7
2        vyE1P
3        KcrkZ
4        AHL4O
         ...  
99995    QywKC
99996    ptvty
99997    09gWa
99998    rqwUm
99999    relB0
Name: id, Length: 99992, dtype: object

'Таблица geo_data_2 с уникальными идентификаторами скважины:'

0        fwXo0
1        WJtFt
2        ovLUW
3        q6cA6
4        WPMUX
         ...  
99995    4GxBu
99996    YKFjq
99997    tKPY3
99998    nmxp2
99999    V9kWn
Name: id, Length: 99992, dtype: object

In [6]:
data0 = data0.drop(['id'], axis=1)
display('Таблица geo_data_0:', data0)

data1 = data1.drop(['id'], axis=1)
display('Таблица geo_data_1:', data1)

data2 = data2.drop(['id'], axis=1)
display('Таблица geo_data_2:', data2)

'Таблица geo_data_0:'

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...
99995,0.971957,0.370953,6.075346,110.744026
99996,1.392429,-0.382606,1.273912,122.346843
99997,1.029585,0.018787,-1.348308,64.375443
99998,0.998163,-0.528582,1.583869,74.040764


'Таблица geo_data_1:'

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...
99995,9.535637,-6.878139,1.998296,53.906522
99996,-10.160631,-12.558096,5.005581,137.945408
99997,-7.378891,-3.084104,4.998651,137.945408
99998,0.665714,-6.152593,1.000146,30.132364


'Таблица geo_data_2:'

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...
99995,-1.777037,1.125220,6.263374,172.327046
99996,-1.261523,-0.894828,2.524545,138.748846
99997,-1.199934,-2.957637,5.219411,157.080080
99998,-2.419896,2.417221,-5.548444,51.795253


## Обучение и проверка модели

### Разбейте данные на обучающую и валидационную выборки в соотношении 75:25

In [8]:
train_data0, valid_data0 = train_test_split(data0, test_size=0.25, random_state=12345)

print('Размеры таблиц:',
      '\ntrain_data =', train_data0.shape,
      '\nvalid_data =', valid_data0.shape
     )

Размеры таблиц: 
train_data = (74985, 4) 
valid_data = (24995, 4)


In [9]:
train_data1, valid_data1 = train_test_split(data1, test_size=0.25, random_state=12345)

print('Размеры таблиц:',
      '\ntrain_data =', train_data1.shape,
      '\nvalid_data =', valid_data1.shape
     )

train_data2, valid_data2 = train_test_split(data2, test_size=0.25, random_state=12345)

print('Размеры таблиц:',
      '\ntrain_data =', train_data2.shape,
      '\nvalid_data =', valid_data2.shape
     )

Размеры таблиц: 
train_data = (74994, 4) 
valid_data = (24998, 4)
Размеры таблиц: 
train_data = (74994, 4) 
valid_data = (24998, 4)


In [11]:
features_train0 = train_data0.drop(['product'], axis=1)
target_train0 = train_data0['product']

features_valid0 = valid_data0.drop(['product'], axis=1)
target_valid0 = valid_data0['product']

print('Размеры таблиц:',
      '\nfeatures_train0 =', features_train0.shape,
      '\ntarget_train0 =', target_train0.shape,
      '\nfeatures_valid0 =', features_valid0.shape,
      '\ntarget_valid0 =', target_valid0.shape
     )

Размеры таблиц: 
features_train0 = (74985, 3) 
target_train0 = (74985,) 
features_valid0 = (24995, 3) 
target_valid0 = (24995,)


In [12]:
features_train1 = train_data1.drop(['product'], axis=1)
target_train1 = train_data1['product']

features_valid1 = valid_data1.drop(['product'], axis=1)
target_valid1 = valid_data1['product']

print('Размеры таблиц:',
      '\nfeatures_train1 =', features_train1.shape,
      '\ntarget_train1 =', target_train1.shape,
      '\nfeatures_valid1 =', features_valid1.shape,
      '\ntarget_valid1 =', target_valid1.shape
     )

features_train2 = train_data2.drop(['product'], axis=1)
target_train2 = train_data2['product']

features_valid2 = valid_data2.drop(['product'], axis=1)
target_valid2 = valid_data2['product']

print('Размеры таблиц:',
      '\nfeatures_train2 =', features_train2.shape,
      '\ntarget_train2 =', target_train2.shape,
      '\nfeatures_valid2 =', features_valid2.shape,
      '\ntarget_valid2 =', target_valid2.shape
     )

Размеры таблиц: 
features_train1 = (74994, 3) 
target_train1 = (74994,) 
features_valid1 = (24998, 3) 
target_valid1 = (24998,)
Размеры таблиц: 
features_train2 = (74994, 3) 
target_train2 = (74994,) 
features_valid2 = (24998, 3) 
target_valid2 = (24998,)


### Обучите модель и сделайте предсказания на валидационной выборке

Для обучения модели подходит линейная регрессия, т.к. другие модели недостаточно предсказуемые.

In [13]:
model0 = LinearRegression()
model0.fit(features_train0, target_train0)

model1 = LinearRegression()
model1.fit(features_train1, target_train1)

model2 = LinearRegression()
model2.fit(features_train2, target_train2)

pred_product0 = model0.predict(features_valid0)
pred_product1 = model1.predict(features_valid1)
pred_product2 = model2.predict(features_valid2)

### Сохраните предсказания и правильные ответы на валидационной выборке

In [14]:
display('Предсказания на валидационной выборке:', pd.Series(pred_product0))

'Предсказания на валидационной выборке:'

0         62.286613
1         70.428178
2         87.349562
3         26.782932
4         34.554999
            ...    
24990     76.559803
24991    122.739744
24992     95.057433
24993     78.944546
24994    115.482999
Length: 24995, dtype: float64

In [15]:
display('Предсказания на валидационной выборке:', pd.Series(pred_product1))
display('Предсказания на валидационной выборке:', pd.Series(pred_product2))

'Предсказания на валидационной выборке:'

0         55.971735
1         54.633633
2         54.674833
3        133.810938
4         85.478312
            ...    
24993      1.194595
24994    109.729474
24995    110.002046
24996    137.901663
24997      0.511335
Length: 24998, dtype: float64

'Предсказания на валидационной выборке:'

0         42.989028
1         95.462292
2        109.408090
3        144.760222
4        103.116807
            ...    
24993    128.238720
24994     70.665701
24995     85.507578
24996    111.319557
24997     75.944310
Length: 24998, dtype: float64

### Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [16]:
pm_pred0 = pred_product0.mean()

print('Средний запас предсказанного сырья для первого региона =', pm_pred0)

mse0 = mean_squared_error(target_valid0, pred_product0)

print('RMSE модели для первого региона =', mse0 ** 0.5)

Средний запас предсказанного сырья для первого региона = 92.42384109947359
RMSE модели для первого региона = 37.716904960382735


In [17]:
pm_pred1 = pred_product1.mean()

print('Средний запас предсказанного сырья для второго региона =', pm_pred1)

mse1 = mean_squared_error(target_valid1, pred_product1)

print('RMSE модели для второго региона =', mse1 ** 0.5)

pm_pred2 = pred_product2.mean()

print('\nСредний запас предсказанного сырья для третьего региона =', pm_pred2)

mse2 = mean_squared_error(target_valid2, pred_product2)

print('RMSE модели для третьего региона =', mse2 ** 0.5)

Средний запас предсказанного сырья для второго региона = 68.98311857983123
RMSE модели для второго региона = 0.8914901390348537

Средний запас предсказанного сырья для третьего региона = 95.11622302076478
RMSE модели для третьего региона = 39.975543264382345


### Проанализируйте результаты.

Вывод: Квадратный корень из средней квадратичной ошибки
<br>RMSE модели для первого региона = 37.716904960382735
<br>RMSE модели для второго региона = 0.8914901390348537
<br>RMSE модели для третьего региона = 39.975543264382345
<br>на эти значения отличается правильный ответ от предсказания. Модель с наименьшим RMSE лучшая, далее будем использовать model0.

In [18]:
pred_product1 = model1.predict(features_valid1)
pred_product2 = model2.predict(features_valid2)

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраните в отдельных переменных

Ставку налога на прибыль возьму за 0, так как это не обговорено в задании, предположу, что долговых обязательств нет.
<br>Объем запасов указан в тыс. баррелей, доход с каждой единицы продукта составляет 450 тыс. рублей. Для удобства расчетов объем запасов оставлю в единицах, а цену за 1 баррель возьму 450000.

In [19]:
points200 = 200

budg = 10000000000

points = 500

price = 450000

### Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе

Для безубыточной работы доход с одной скважины должен составлять не меньше 10 млрд / 200 скважин = 5 млрд, объем запасов тогда должен быть не меньше (10 млрд / 200 скважин) / 450 тыс = 111,11.

In [20]:
product_profit = (budg / points200) / price
print('Необходимый средний объем запасов в регионе для безубыточной деятельности =', product_profit)

pm0 = data0['product'].mean()
pm1 = data1['product'].mean()
pm2 = data2['product'].mean()

print('Средний объем запасов в регионе 0 =', pm0,
      '\nСредний объем запасов в регионе 1 =', pm1,
      '\nСредний объем запасов в регионе 2 =', pm2
     )

Необходимый средний объем запасов в регионе для безубыточной деятельности = 111.11111111111111
Средний объем запасов в регионе 0 = 92.49916597893444 
Средний объем запасов в регионе 1 = 68.82414772665173 
Средний объем запасов в регионе 2 = 94.99876686768079


### Напишите выводы по этапу подготовки расчёта прибыли

Для расчета прибыли и рисков буду ориентироваться на значение объемов запасов безубыточной деятельности >= 111.11111111111111

## Расчёт прибыли и рисков 


### Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [25]:
def profit(pred, real, pc200, p, b):

    pred = pd.Series(pred)
    real = pd.Series(real)
    
    pred.index = real.index
    
    pred_sorted = pred.sort_values(ascending=False)

    real_sorted = real.sort_values(ascending=False)

    selected = real_sorted.loc[pred_sorted.index][:pc200]
    
    return selected.sum() * p - b

### Выберите скважины с максимальными значениями предсказаний

In [28]:
mse00 = mean_squared_error(target_valid0, pred_product0)
print('RMSE модели для первого региона =', mse00 ** 0.5)

mse11 = mean_squared_error(target_valid1, pred_product1)
print('RMSE модели для второго региона =', mse11 ** 0.5)

mse22 = mean_squared_error(target_valid2, pred_product2)
print('RMSE модели для третьего региона =', mse22 ** 0.5)

RMSE модели для первого региона = 37.716904960382735
RMSE модели для второго региона = 0.8914901390348537
RMSE модели для третьего региона = 39.975543264382345


In [29]:
pred0 = pd.DataFrame(pred_product0).reset_index()
data0_id = pd.DataFrame(data0_id).reset_index()
max0 = pd.merge(pred0, data0_id, on='index')
max0 = max0.rename(columns = {0: 'pred_product'}) 
max0_id = max0.pivot_table(index='id', values='pred_product', aggfunc='max').sort_values('pred_product', ascending=False)[:points200]

pred1 = pd.DataFrame(pred_product1).reset_index()
data1_id = pd.DataFrame(data1_id).reset_index()
max1 = pd.merge(pred1, data1_id, on='index')
max1 = max1.rename(columns = {0: 'pred_product'}) 
max1_id = max1.pivot_table(index='id', values='pred_product', aggfunc='max').sort_values('pred_product', ascending=False)[:points200]


pred2 = pd.DataFrame(pred_product2).reset_index()
data2_id = pd.DataFrame(data2_id).reset_index()
max2 = pd.merge(pred2, data2_id, on='index')
max2 = max2.rename(columns = {0: 'pred_product'}) 
max2_id = max2.pivot_table(index='id', values='pred_product', aggfunc='max').sort_values('pred_product', ascending=False)[:points200]

display('Скважины с максимальными значениями предсказаний в регионе 0:', max0_id)
display('Скважины с максимальными значениями предсказаний в регионе 1:', max1_id)
display('Скважины с максимальными значениями предсказаний в регионе 2:', max2_id)

'Скважины с максимальными значениями предсказаний в регионе 0:'

,pred_product
id,
5lB0f,173.769148
NpVmK,172.612158
Ci5hu,172.558475
wCxwS,170.986975
JU9Cf,169.739879
...,...
Np4xV,147.666815
0rRkD,147.665931
9fz3b,147.655987


'Скважины с максимальными значениями предсказаний в регионе 1:'

,pred_product
id,
v7ppj,140.353099
58t1Q,139.976751
ECSVF,139.844652
eofq1,139.758002
1Jjlh,139.663321
...,...
qEsKj,138.401607
nO6DK,138.401299
EBClQ,138.400850


'Скважины с максимальными значениями предсказаний в регионе 2:'

,pred_product
id,
PqOj4,174.374933
WXorA,170.939857
Wthdm,166.709877
crng6,166.569756
cLSl3,166.036689
...,...
QWHu7,143.753122
CcTKY,143.718548
Tsf0T,143.704474


### Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям

In [30]:
print('Целевое значение объёма сырья в регионе 0 =', max0_id['pred_product'].sum())
print('Целевое значение объёма сырья в регионе 1 =', max1_id['pred_product'].sum())
print('Целевое значение объёма сырья в регионе 2 =', max2_id['pred_product'].sum())

Целевое значение объёма сырья в регионе 0 = 30804.345930358257
Целевое значение объёма сырья в регионе 1 = 27744.434181599772
Целевое значение объёма сырья в регионе 2 = 29988.979470532056


### Рассчитайте прибыль для полученного объёма сырья

In [32]:
net_profit0 = max0_id['pred_product'].sum() * price - budg
print('Прибыль для полученного объёма сырья в регионе 0 =', net_profit0)

net_profit1 = max1_id['pred_product'].sum() * price - budg
print('Прибыль для полученного объёма сырья в регионе 1 =', net_profit1)

net_profit2 = max2_id['pred_product'].sum() * price - budg
print('Прибыль для полученного объёма сырья в регионе 2 =', net_profit2)

Прибыль для полученного объёма сырья в регионе 0 = 3861955668.661215
Прибыль для полученного объёма сырья в регионе 1 = 2484995381.719898
Прибыль для полученного объёма сырья в регионе 2 = 3495040761.7394257


## Посчитайте риски и прибыль для каждого региона

### Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

In [36]:
pred_product0 = pd.Series(pred_product0)
pred_product0.index = target_valid0.index

pred_product1 = pd.Series(pred_product1)
pred_product1.index = target_valid1.index

pred_product2 = pd.Series(pred_product2)
pred_product2.index = target_valid2.index

In [37]:
state = np.random.RandomState(12345)

value0 = []
value1 = []
value2 = []

for i in range(1000):
    
    pred_subsample0 = pred_product0.sample(n=points, replace=True, random_state=state) #frac=points, 
    target_subsample0 = target_valid0.loc[pred_subsample0.index]
    value0.append(profit(pred_subsample0, target_subsample0, points200, price, budg))

    pred_subsample1 = pred_product1.sample(n=points, replace=True, random_state=state) #frac=points, 
    target_subsample1 = target_valid1.loc[pred_subsample1.index]
    value1.append(profit(pred_subsample1, target_subsample1, points200, price, budg))

    pred_subsample2 = pred_product2.sample(n=points, replace=True, random_state=state) #frac=points, 
    target_subsample2 = target_valid2.loc[pred_subsample2.index]
    value2.append(profit(pred_subsample2, target_subsample2, points200, price, budg))    

### Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль

In [43]:
value0 = pd.Series(value0)
value1 = pd.Series(value1)
value2 = pd.Series(value2)

In [45]:
profit_mean0 = value0.mean()
print('Средняя прибыль для региона 0 =', profit_mean0)

profit_mean1 = value1.mean()
print('Средняя прибыль для региона 1 =', profit_mean1)

profit_mean2 = value2.mean()
print('Средняя прибыль для региона 2 =', profit_mean2)

Средняя прибыль для региона 0 = 464473127.68013746
Средняя прибыль для региона 1 = 525400905.34057534
Средняя прибыль для региона 2 = 338878364.0956245


In [46]:
lower0 = value0.quantile(0.025)
higher0 = value0.quantile(0.975)
print('95%-ый доверительный интервал для региона 0:', lower0, '-', higher0)

lower1 = value1.quantile(0.025)
higher1 = value1.quantile(0.975)
print('95%-ый доверительный интервал для региона 1:', lower1, '-', higher1)

lower2 = value2.quantile(0.025)
higher2 = value2.quantile(0.975)
print('95%-ый доверительный интервал для региона 2:', lower2, '-', higher2)

95%-ый доверительный интервал для региона 0: -52044802.09013366 - 967200461.7868712
95%-ый доверительный интервал для региона 1: 137355241.66832852 - 935450630.7871925
95%-ый доверительный интервал для региона 2: -197662818.63274705 - 891519006.8475438


In [48]:
print('Риск убытков для региона 0 = {:.2%}'.format(
                                     value0[value0 < 0].count() / len(value0)
                                                   )
      )
print('Риск убытков для региона 1 = {:.2%}'.format(
                                    value1[value1 < 0].count() / len(value1)
                                                  )
     )
print('Риск убытков для региона 2 = {:.2%}'.format(
                                    value2[value2 < 0].count() / len(value2)
                                                  )
     )

Риск убытков для региона 0 = 4.10%
Риск убытков для региона 1 = 0.80%
Риск убытков для региона 2 = 11.90%


### Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор

Вероятность убытков меньше 2.5% в регионе 1, средняя прибыль для региона 1 составялет 525400905.34057534, что выше средней прибыли в регионах 0 и 2. Т.е. после проведенного анализа и расчетов делаю вывод что регион 1 принесет наибольшую прибыль.